<a href="https://colab.research.google.com/github/Zernach/BetterReads-DS/blob/master/How_to_Generate_Book_Recommendations_NLP_Hybrid_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Statements

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
!pip install squarify
import squarify
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import ast
from spacy.tokenizer import Tokenizer
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


# Load, Modify, and Check the Datasets

In [0]:
extracted_by_jose = pd.read_csv('https://ryan.zernach.com/wp-content/uploads/2020/05/goodreads_books_ver2.csv')
extracted_by_michael = pd.read_csv('https://ryan.zernach.com/wp-content/uploads/2020/05/output.csv')
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

# Modifications & Adjustments
extracted_by_jose['authors'] = extracted_by_jose['authors'].apply(ast.literal_eval)
extracted_by_jose['age'] = (2020 - extracted_by_jose['original_publication_year'])
extracted_by_jose['age'] = extracted_by_jose['age'].fillna(-1)
extracted_by_jose['age'] = extracted_by_jose['age'].apply(int)
extracted_by_jose['age'] = extracted_by_jose['age'].replace(-1, np.nan)

extracted_by_michael['goodreads_book_id'] = extracted_by_michael['id']
extracted_by_michael['isbn13'] = extracted_by_michael['isbn13'].fillna(-1)
extracted_by_michael['isbn13'] = extracted_by_michael['isbn13'].apply(int)
extracted_by_michael['isbn13'] = extracted_by_michael['isbn13'].astype(str)
extracted_by_michael['isbn13'] = extracted_by_michael['isbn13'].replace('-1', np.nan)

In [4]:
extracted_by_jose.head(1)

,goodreads_book_id,title,authors,description,isbn,isbn13,publication_year,language_code,publisher,average_rating,num_pages,original_title,original_publication_year,age
0,11324,Under the Net,[Iris Murdoch],"<b>Jake Donaghue, garrulous artist, meets Hugo...",0140014454,9780140014457,1982.0,eng,Penguin Books Ltd,3.76,252,Under The Net,1954.0,66.0


In [5]:
extracted_by_michael.head(1)

,id,title,isbn,isbn13,asin,kindle_asin,marketplace_id,country_code,image_url,small_image_url,publication_year,publication_month,publication_day,publisher,language_code,is_ebook,description,work,work_work,work_id,work_books_count,work_best_book_id,work_reviews_count,work_ratings_sum,work_ratings_count,work_text_reviews_count,work_original_publication_year,work_original_publication_month,work_original_publication_day,work_original_title,work_original_language_id,work_media_type,work_rating_dist,work_desc_user_id,work_default_chaptering_book_id,work_default_description_language_code,average_rating,num_pages,format,edition_information,ratings_count,text_reviews_count,url,link,authors,authors_authors,authors_author,authors_id,authors_name,authors_role,authors_image_url,authors_small_image_url,authors_link,authors_average_rating,authors_ratings_count,authors_text_reviews_count,public_document,public_document_public_document,public_document_id,public_document_document_url,goodreads_book_id
0,1162022,On the Jellicoe Road,0670070297,9780670070299,NaN,B00AMH0S8A,A1F83G8C2ARO7P,GB,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,2006.0,8.0,28.0,Penguin Australia,eng,False,I'm dreaming of the boy in the tree. I tell hi...,\n,\n,6479100,37,1162022,121977,168383,40624,6185,2006.0,NaN,NaN,On the Jellicoe Road,NaN,book,5:20011|4:11237|3:5957|2:2090|1:1329|total:40624,171430.0,NaN,NaN,4.14,290.0,Paperback,1st edition,33421,3771,https://www.goodreads.com/book/show/1162022.On...,https://www.goodreads.com/book/show/1162022.On...,\n,\n,\n,47104,Melina Marchetta,NaN,\nhttps://images.gr-assets.com/authors/1277655...,\nhttps://images.gr-assets.com/authors/1277655...,https://www.goodreads.com/author/show/47104.Me...,4.06,159449,19650,NaN,NaN,NaN,NaN,1162022


In [6]:
books.head(1)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [21]:
ratings.head(10)

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
5,2,26,4
6,2,315,3
7,2,33,4
8,2,301,5
9,2,2686,5


# Dataset Merging

In [0]:
books = books[['book_id', 'goodreads_book_id']]
merged = extracted_by_michael.merge(books, how='left', on='goodreads_book_id')
authors_corrections = extracted_by_jose[['authors', 'goodreads_book_id']]
merged = merged.merge(authors_corrections, how='left', on='goodreads_book_id')

# Language Code Filtering (English Only)
merged = merged[(merged['language_code'] == 'eng') | (merged['language_code'] == 'en-US') | (merged['language_code'] == 'en-GB') | (merged['language_code'] == 'en-CA') | (merged['language_code'] == 'en')]

# Work Media Type Filtering (Books only)
merged = merged[merged['work_media_type'] == 'book']

In [9]:
columns = ['title', 'isbn', 'isbn13', 'publisher', 'is_ebook', 'description', 'goodreads_book_id', 'authors_y']
condensed = merged[columns]
condensed

,title,isbn,isbn13,publisher,is_ebook,description,goodreads_book_id,authors_y
0,On the Jellicoe Road,0670070297,9780670070299,Penguin Australia,False,I'm dreaming of the boy in the tree. I tell hi...,1162022,[Melina Marchetta]
1,"I've Got You Under My Skin (Under Suspicion, #1)",147674906X,9781476749068,Simon Schuster,False,When Laurie Moran's husband was brutally murde...,18143968,"[Mary Higgins Clark, Alafair Burke]"
2,The Orange Girl,0753819929,9780753819920,NaN,False,'My father died eleven years ago. I was only f...,25403,"[Jostein Gaarder, James Anderson]"
3,The Informers,0330339184,9780330339186,MacMillan,False,"Set in Los Angeles, in the recent past. The bi...",9914,[Bret Easton Ellis]
4,Out,1400078377,9781400078370,Vintage,False,Natsuo Kirino's novel tells a story of random ...,25365,"[Natsuo Kirino, Stephen Snyder]"
...,...,...,...,...,...,...,...,...
9994,"Soul Eater, Vol. 02 (Soul Eater, #2)",0759530483,9780759530485,Yen Press,False,The remedial assignment continues as Soul and ...,6448482,[Atsushi Ohkubo]
9995,"The Way of Kings (The Stormlight Archive, #1)",0765326353,9780765326355,Tor Books,False,"Speak again the ancient oaths,Life before deat...",7235533,[Brandon Sanderson]
9996,"Reaper Man (Discworld, #11; Death, #2)",0552152951,9780552152952,Corgi,False,'Death has to happen. That's what bein' alive ...,34517,[Terry Pratchett]
9998,"Just Ella (The Palace Chronicles, #1)",1416936491,9781416936497,Simon Pulse,False,Being a princess isn't all that...You've heard...,162719,[Margaret Peterson Haddix]


# Content Based Model 
Testing a content-based recommendation system on a dataset of 12k+ books. The main question in this experiment:

"Will a recommendation based on descriptions provide better results than the details laten in a collaborative filtering 
system?"

In [22]:
from bs4 import BeautifulSoup
def get_text(text):
    """Extracts text from html"""
    if type(text) == float:
        return "None"
    soup = BeautifulSoup(text)
    return soup.text

condensed['description'] = condensed['description'].apply(get_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [0]:
# Visualize all words to decide whether or not to extend stop-words

def get_simple_tokens(corpus):
    tokens = []
    doc = nlp(corpus)

    for token in doc:
        if token.text not in stop_words and token.is_punct == False:
            tokens.append(token.text)
    
    return tokens

In [0]:
# Extend stop words
stop_words = nlp.Defaults.stop_words.union(["new", "book", "author",
                                            "story", "life", "work", "best",
                                            "edition", "readers", "includes",
                                            "including", "provides", "information"])

In [17]:
# Visualize top words
wc = count(tokens)
top_20 = wc[wc["rank"] <= 20]

squarify.plot(sizes=top_20["pct_total"], label=top_20["word"], alpha=0.8)
plt.axis("off")
plt.show()

NameError: ignored

### Vectorize Descriptions using TFIDF

In [0]:
tfidf = TfidfVectorizer(
                  analyzer='word', 
                  ngram_range=(1, 2), min_df=15,
                  max_df=.85, tokenizer=get_simple_tokens)
tfidf_matrix = tfidf.fit_transform(books['desc'])
dtm = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf.get_feature_names())

In [20]:
nn = NearestNeighbors(metric='cosine')
nn.fit(dtm)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [0]:
ready_player_one = ['Immersing himself in a mid-21st-century '
                                     'technological virtual utopia to escape '
                                     'an ugly real world of famine, poverty '
                                     'and disease, Wade Watts joins an '
                                     'increasingly violent effort to solve a '
                                     "series of puzzles by the virtual world's "
                                     'super-wealthy creator, who has promised '
                                     'that the winner will be his heir. (This '
                                     'book was previously listed in Forecast.)']

the_martian = ["Nominated as one of America’s best-loved novels by PBS’s The Great American Read Six days ago, astronaut Mark Watney became one of the first people to walk on Mars. Now, he's sure he'll be the first person to die there. After a dust storm nearly kills him and forces his crew to evacuate while thinking him dead, Mark finds himself stranded and completely alone with no way to even signal Earth that he’s alive—and even if he could get word out, his supplies would be gone long before a rescue could arrive. Chances are, though, he won't have time to starve to death. The damaged machinery, unforgiving environment, or plain-old human error are much more likely to kill him first. But Mark isn't ready to give up yet. Drawing on his ingenuity, his engineering skills—and a relentless, dogged refusal to quit—he steadfastly confronts one seemingly insurmountable obstacle after the next. Will his resourcefulness be enough to overcome the impossible odds against him?"]

In [0]:
# Get nearest neighbor for book description
example_book = tfidf.transform(ready_player_one)
neighbors = nn.kneighbors(example_book.todense(), n_neighbors=10)

In [24]:
for index in neighbors[1]:
    print(books['title'].iloc[index])

10820                            Ready Player One
8105     Organizational Behavior and Virtual Work
595                                Slaughtermatic
8042                       Managing Without Walls
9082                Understanding Virtual Reality
2878                       Escape Book (volume 1)
829                           Algorithmic Puzzles
793                     Jumbo Sudoku Hard as Heck
11946                                EDENS ZERO 5
6472                    Jumbo Pad of Puzzling Fun
Name: title, dtype: object


In [25]:
books.iloc[neighbors[1][0]]

,id,title,authors,publisher,publishedDate,description,isbn,pageCount,categories,thumbnail,smallThumbnail,language,webReaderLink,textSnippet,isEbook,averageRating,maturityRating,ratingsCount,subtitle,index,desc
10820,FY_HWAcm10MC,Ready Player One,Ernest Cline,Broadway Books,2011-08-16,Nominated as one of America’s best-loved novel...,0307887456,384.0,Fiction,http://books.google.com/books/content?id=FY_HW...,http://books.google.com/books/content?id=FY_HW...,en,http://play.google.com/books/reader?id=FY_HWAc...,NaN,1.0,4.0,NOT_MATURE,3450.0,NaN,0.0,nominated as one of america’s best-loved novel...
8105,etBBDgAAQBAJ,Organizational Behavior and Virtual Work,Arvind K. Birdie,CRC Press,2017-03-03,With the globalization of work and technologic...,1771882840,296.0,Business & Economics,http://books.google.com/books/content?id=etBBD...,http://books.google.com/books/content?id=etBBD...,en,http://play.google.com/books/reader?id=etBBDgA...,NaN,1.0,NaN,NOT_MATURE,NaN,Concepts and Analytical Approaches,0.0,with the globalization of work and technologic...
595,o_vEPgAACAAJ,Slaughtermatic,Steve Aylett,Phoenix,1999,This parody of a future driven by cyber-techno...,9780753807408,181.0,NaN,NaN,NaN,en,http://play.google.com/books/reader?id=o_vEPgA...,NaN,0.0,4.0,NOT_MATURE,2.0,NaN,NaN,this parody of a future driven by cyber-techno...
8042,WQBcAAAACAAJ,Managing Without Walls,Colleen Garton,Mc PressLlc,2006,As companies worldwide are moving more and mor...,9781583470626,420.0,Business & Economics,http://books.google.com/books/content?id=WQBcA...,http://books.google.com/books/content?id=WQBcA...,en,http://play.google.com/books/reader?id=WQBcAAA...,NaN,0.0,NaN,NOT_MATURE,NaN,"Maximize Success with Virtual, Global, and Cro...",0.0,as companies worldwide are moving more and mor...
9082,7ssd_st8kjMC,Understanding Virtual Reality,William R. Sherman,Morgan Kaufmann,2003,Exploring the practical applications of virtua...,9781558603530,582.0,Computers,http://books.google.com/books/content?id=7ssd_...,http://books.google.com/books/content?id=7ssd_...,en,http://play.google.com/books/reader?id=7ssd_st...,NaN,0.0,NaN,NOT_MATURE,NaN,"Interface, Application, and Design",0.0,exploring the practical applications of virtua...
2878,2r8SzAEACAAJ,Escape Book (volume 1),Alain Puyssegur,NaN,2020-04-21,Solve puzzles and riddles with your favorite M...,9781524858865,96.0,Juvenile Fiction,NaN,NaN,en,http://play.google.com/books/reader?id=2r8SzAE...,NaN,0.0,NaN,NOT_MATURE,NaN,The Cursed Temple,NaN,solve puzzles and riddles with your favorite m...
829,UcgMWDfca0EC,Algorithmic Puzzles,Anany Levitin,OUP USA,2011-10-14,Algorithmic puzzles are puzzles involving well...,0199740445,257.0,Computers,http://books.google.com/books/content?id=UcgMW...,http://books.google.com/books/content?id=UcgMW...,en,http://play.google.com/books/reader?id=UcgMWDf...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,algorithmic puzzles are puzzles involving well...
793,2HlOPgAACAAJ,Jumbo Sudoku Hard as Heck,Sudoku,Time Home Entertainment,2009-10-27,"The next instalment in the popular "" Jumbo Sud...",9781603208246,240.0,Games,http://books.google.com/books/content?id=2HlOP...,http://books.google.com/books/content?id=2HlOP...,en,http://play.google.com/books/reader?id=2HlOPgA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,"the next instalment in the popular "" jumbo sud..."
11946,y8W-DwAAQBAJ,EDENS ZERO 5,Hiro Mashima,Kodansha America LLC,2019-12-10,THE HEART OF A HERMIT In order to restore the ...,1646591488,192.0,Comics & Graphic Novels,http://books.google.com/books/content?id=y8W-D...,http://books.google.com/books/content?id=y8W-D...,en,http://play.google.com/books/reader?id=y8W-DwA...,NaN,1.0,NaN,NOT_MATURE,NaN,NaN,0.0,the heart of a hermit in order to restore the ...
6472,dp_ojwEACAAJ,Jumbo Pad of Puzzling Fun,Highlights,Highlights,2016-08-23,Nobody does puzzles like Highlights™! With bri...,9781629796178,256.0,NaN,http://books.google.com/books/content?id=dp_oj...,http://books.google.com/books/content?id=dp_oj...,en,http://pla

In [0]:
martian_book = tfidf.transform(the_martian)
martian_neighbors = nn.kneighbors(martian_book.todense(), n_neighbors=10)
books.iloc[martian_neighbors[1][0]]

In [27]:
book_index = 12640

example = [books['desc'][book_index]]
transformed = tfidf.transform(example)
distances, indices = nn.kneighbors(transformed.todense())

print(books['title'].iloc[book_index])
books.iloc[indices.tolist()[0]]

Martha Monkey


,id,title,authors,publisher,publishedDate,description,isbn,pageCount,categories,thumbnail,smallThumbnail,language,webReaderLink,textSnippet,isEbook,averageRating,maturityRating,ratingsCount,subtitle,index,desc
12640,0RCdGgAACAAJ,Martha Monkey,Enid Blyton,HarperCollins Children's Books,2008,It's Martha Monkey's birthday and she's got a ...,9780007269228,32.0,Monkeys,http://books.google.com/books/content?id=0RCdG...,http://books.google.com/books/content?id=0RCdG...,en,http://play.google.com/books/reader?id=0RCdGgA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,0.0,it's martha monkey's birthday and she's got a ...
3205,SE7tyQEACAAJ,Apocalypse,Jamie Smart,Phoenix Presents,2019-11,Everythings gone wrong! The Woods normally ech...,9781910989265,64.0,NaN,http://books.google.com/books/content?id=SE7ty...,http://books.google.com/books/content?id=SE7ty...,en,http://play.google.com/books/reader?id=SE7tyQE...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,everythings gone wrong! the woods normally ech...
5218,VRdqYgEACAAJ,"Happy Birthday, Davy!",Brigitte Weninger,North South Books,2011,Take these rough and tumble tiny books with yo...,9780735840263,24.0,Birthdays,http://books.google.com/books/content?id=VRdqY...,http://books.google.com/books/content?id=VRdqY...,en,http://play.google.com/books/reader?id=VRdqYgE...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,take these rough and tumble tiny books with yo...
8981,rsvcIuKPp7AC,The Quest for the Tree Kangaroo,Sy Montgomery,Houghton Mifflin Harcourt,2009-10-19,"It looks like a bear, but isn’t one. It climbs...",9780547529882,80.0,Young Adult Nonfiction,http://books.google.com/books/content?id=rsvcI...,http://books.google.com/books/content?id=rsvcI...,en,http://play.google.com/books/reader?id=rsvcIuK...,NaN,1.0,3.5,NOT_MATURE,18.0,An Expedition to the Cloud Forest of New Guinea,0.0,"it looks like a bear, but isn’t one. it climbs..."
10715,18NXPgAACAAJ,The Secretary of Dreams,Stephen King,NaN,2010,Collects six of the author's early tales of te...,9781587671852,260.0,Fiction,http://books.google.com/books/content?id=18NXP...,http://books.google.com/books/content?id=18NXP...,en,http://play.google.com/books/reader?id=18NXPgA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,0.0,collects six of the author's early tales of te...


In [28]:
distances

array([[0.        , 0.7022254 , 0.78597611, 0.79336576, 0.79657545]])

In [0]:
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
def get_cos_recs(index):
    sim_scores = list(enumerate(cos_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return books['title'].iloc[book_indices]

In [0]:
cos_books = get_cos_recs(12640)

In [0]:
line = 0
for word in books['description'].iloc[12640].split():
    if line > 5:
        print(word)
        line = 0
    else:
        print(word, end=' ')
        line += 1

It's Martha Monkey's birthday and she's got
a brand new pair of roller skates,
but where will they take her? And
will Noddy be able to keep up?
Martha Monkey means well but often does
the wrong thing. Can Noddy save her
from the Goblins on her birthday? 

In [0]:
books.loc[cos_books.index]

,id,title,authors,publisher,publishedDate,description,isbn,pageCount,categories,thumbnail,smallThumbnail,language,webReaderLink,textSnippet,isEbook,averageRating,maturityRating,ratingsCount,subtitle,index,desc
3205,SE7tyQEACAAJ,Apocalypse,Jamie Smart,Phoenix Presents,2019-11,Everythings gone wrong! The Woods normally ech...,9781910989265,64.0,NaN,http://books.google.com/books/content?id=SE7ty...,http://books.google.com/books/content?id=SE7ty...,en,http://play.google.com/books/reader?id=SE7tyQE...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,everythings gone wrong! the woods normally ech...
5218,VRdqYgEACAAJ,"Happy Birthday, Davy!",Brigitte Weninger,North South Books,2011,Take these rough and tumble tiny books with yo...,9780735840263,24.0,Birthdays,http://books.google.com/books/content?id=VRdqY...,http://books.google.com/books/content?id=VRdqY...,en,http://play.google.com/books/reader?id=VRdqYgE...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,take these rough and tumble tiny books with yo...
8981,rsvcIuKPp7AC,The Quest for the Tree Kangaroo,Sy Montgomery,Houghton Mifflin Harcourt,2009-10-19,"It looks like a bear, but isn’t one. It climbs...",9780547529882,80.0,Young Adult Nonfiction,http://books.google.com/books/content?id=rsvcI...,http://books.google.com/books/content?id=rsvcI...,en,http://play.google.com/books/reader?id=rsvcIuK...,NaN,1.0,3.5,NOT_MATURE,18.0,An Expedition to the Cloud Forest of New Guinea,0.0,"it looks like a bear, but isn’t one. it climbs..."
10715,18NXPgAACAAJ,The Secretary of Dreams,Stephen King,NaN,2010,Collects six of the author's early tales of te...,9781587671852,260.0,Fiction,http://books.google.com/books/content?id=18NXP...,http://books.google.com/books/content?id=18NXP...,en,http://play.google.com/books/reader?id=18NXPgA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,0.0,collects six of the author's early tales of te...
4384,AiIyzQEACAAJ,Kai and the Monkey King,Joe Todd-Stanton,Brownstone's Mythical Collecti,2020-10-06,When Kai grows tired of her bookish mum not be...,9781912497522,56.0,Juvenile Fiction,http://books.google.com/books/content?id=AiIyz...,http://books.google.com/books/content?id=AiIyz...,en,http://play.google.com/books/reader?id=AiIyzQE...,NaN,0.0,NaN,NOT_MATURE,NaN,Brownstone's Mythical Collection 3,NaN,when kai grows tired of her bookish mum not be...
5255,7FiwHAAACAAJ,The Case of the Missing Birthday Party,Joanne Rocklin,Cartwheel Books,1997-01-01,Liz the Whiz & Co. help a neighbor find her wa...,9780590673594,48.0,Birthday parties,http://books.google.com/books/content?id=7FiwH...,http://books.google.com/books/content?id=7FiwH...,en,http://play.google.com/books/reader?id=7FiwHAA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,NaN,liz the whiz & co. help a neighbor find her wa...
6126,zGCxDAAAQBAJ,Sticks and Stones,Lew French,Gibbs Smith,2016-08-09,Lew French is a master stonemason who finds an...,1423637593,160.0,Architecture,http://books.google.com/books/content?id=zGCxD...,http://books.google.com/books/content?id=zGCxD...,en,http://play.google.com/books/reader?id=zGCxDAA...,NaN,1.0,NaN,NOT_MATURE,NaN,NaN,0.0,lew french is a master stonemason who finds an...
6144,7ZazDAAAQBAJ,The Quiz Book about Me ... and You!,Helaine Becker,Scholastic Canada,2015-10,Who do you think you are? What makes you tick?...,1443143006,96.0,Children's questions and answers,http://books.google.com/books/content?id=7ZazD...,http://books.google.com/books/content?id=7ZazD...,en,http://play.google.com/books/reader?id=7ZazDAA...,NaN,0.0,NaN,NOT_MATURE,NaN,NaN,0.0,who do you think you are? what makes you tick?...
62,HubqPAAACAAJ,The City Tavern Cookbook,Walter Staib,Running PressBook Pub,2009,Collects more than three hundred recipes from ...,9780762434176,392.0,Cooking,http://books.google.com/books/content?id=HubqP...,http://books.google.com/books/content?id=HubqP...,en,http://play.google.com/books/reader?id=HubqPAA...,NaN,0.0,NaN,NOT_MATURE,NaN,Recipes from the Birthplace of American Cuisine,NaN,collects more than three hundred recipes from ...
2871,rnrNDwA

In [0]:
line = 0
for word in books['description'].iloc[3205].split():
    if line > 5:
        print(word)
        line = 0
    else:
        print(word, end=' ')
        line += 1

Everythings gone wrong! The Woods normally echo
with the sounds of Bunny and Monkey
fighting, but this time they are facing
their greatest danger yetthe actual apocalypse! Bunny
and Monkey might need to put their
differences aside and work together to stop
it from destroying absolutely everything! 

A few things to do in this notebook:
- Find a good way to get as much meaning from the description as possible
- Create some sort of vector, tfidf may not work that well here, needs to be seen
- Try the recommendation system from the notebook, and try nearest neighbors